In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil

import tensorflow as tf

from tensorflow.keras.models import Sequential, load_model, Model
from tensorflow.keras.layers import Dense, Flatten, Input, concatenate
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import SGD, Adam, RMSprop

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import tensorflow.keras.utils as utils

from datetime import datetime

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from google.colab import drive
import zipfile

from tensorflow.keras.applications import Xception, ResNet50, InceptionV3, MobileNet
from tensorflow.keras.layers import GlobalAveragePooling2D

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.initializers import he_normal


In [ ]:
drive.mount('/content/gdirve')


Mounted at /content/gdirve


In [ ]:
ROOT_DIR = "/content"
DATA_ROOT_DIR = os.path.join(ROOT_DIR, "car_damage_type_2")

shutil.copy(os.path.join("/content/gdirve/MyDrive/FINAL PROJECT" , "car_damage_type_2.zip"), ROOT_DIR)

with zipfile.ZipFile(os.path.join(ROOT_DIR, "car_damage_type_2.zip"), "r") as target_file:
    target_file.extractall(DATA_ROOT_DIR)

In [ ]:
DATADATA_ROOT_DIR = os.path.join(DATA_ROOT_DIR,"car_damage_type")

TRAIN_DATA_ROOT_DIR = os.path.join(DATADATA_ROOT_DIR,"train")
TEST_DATA_ROOT_DIR = os.path.join(DATADATA_ROOT_DIR,'test')

In [ ]:
IMG_WIDTH = 100
IMG_HEIGHT = 100
train_data_gen = ImageDataGenerator(rescale=1./255)
test_data_gen = ImageDataGenerator(rescale=1./255)
train_generator = train_data_gen.flow_from_directory(TRAIN_DATA_ROOT_DIR,
                                                     batch_size = 32,
                                                     color_mode = "rgb",
                                                     class_mode = "categorical",
                                                     target_size = (IMG_WIDTH, IMG_HEIGHT))

test_generator = train_data_gen.flow_from_directory(TEST_DATA_ROOT_DIR,
                                                     batch_size= 32,
                                                     color_mode = "rgb",
                                                     class_mode = "categorical",
                                                     target_size = (IMG_WIDTH, IMG_HEIGHT))


Found 46312 images belonging to 4 classes.
Found 10576 images belonging to 4 classes.


In [ ]:
pre_model = pre_trained_model = ResNet50(weights = "imagenet", include_top=False,
                             input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))

pre_trained_model.summary()

94765736/94765736 [==============================] - 0s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 100, 100, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 106, 106, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 50, 50, 64)   9472        ['conv1_pad[0][0]']              
                                                                                                  
 conv1_bn (BatchNormalizat

In [ ]:
model = Sequential()

model.add(pre_trained_model)

model.add(GlobalAveragePooling2D())  # Flatten 대신 사용

model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(4, activation="softmax"))

model.summary()


model.compile(loss="categorical_crossentropy",
             optimizer=Adam(1e-5), metrics = ["acc"])

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 4, 4, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 64)                131136    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_1 (Dense)             (None, 128)               8320      
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                        

In [ ]:
start_time = datetime.now()

result = model.fit(train_generator, epochs= 30, validation_data=test_generator)

end_time = datetime.now()

Epoch 1/30
1448/1448 [==============================] - 655s 416ms/step - loss: 1.3328 - acc: 0.3966 - val_loss: 1.1094 - val_acc: 0.5162
Epoch 2/30
1448/1448 [==============================] - 586s 404ms/step - loss: 1.1095 - acc: 0.5223 - val_loss: 1.0054 - val_acc: 0.5660
Epoch 3/30
1448/1448 [==============================] - 614s 424ms/step - loss: 0.9978 - acc: 0.5781 - val_loss: 0.9474 - val_acc: 0.6066
Epoch 4/30
1448/1448 [==============================] - 559s 386ms/step - loss: 0.8976 - acc: 0.6259 - val_loss: 0.9132 - val_acc: 0.6388
Epoch 5/30
1448/1448 [==============================] - 553s 382ms/step - loss: 0.7966 - acc: 0.6712 - val_loss: 0.8975 - val_acc: 0.6599
Epoch 6/30
1448/1448 [==============================] - 553s 382ms/step - loss: 0.6949 - acc: 0.7180 - val_loss: 0.9056 - val_acc: 0.6687
Epoch 7/30
1448/1448 [==============================] - 591s 408ms/step - loss: 0.5945 - acc: 0.7636 - val_loss: 0.9779 - val_acc: 0.6741
Epoch 8/30
1448/1448 [============

KeyboardInterrupt: ignored